In [29]:
! pip install python==3.11
! pip install gradio==4.44.0
! pip install ibm-watsonx-ai==1.1.2
! pip install langchain==0.2.11
! pip install langchain-community==0.2.10
! pip install langchain-ibm==0.1.11
! pip install chromadb==0.4.24
! pip install pypdf==4.3.1
! pip install pydantic==2.9.1
! pip install huggingface_hub==0.23.0

ERROR: Could not find a version that satisfies the requirement python==3.11 (from versions: none)
ERROR: No matching distribution found for python==3.11


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
error: failed-wheel-build-for-install

Failed to build installable wheels for some pyproject.toml based projects

chroma-hnswlib



  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached chroma-hnswlib-0.7.3.tar.gz (31 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached posthog-7.0.1-py3-none-any.whl.metadata (6.0 kB)
  Using cached pulsar_client-3.8.0-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached onnxruntime-1.23.2-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached opentelemetry_api-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.38.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.59b0-py3-none-any.whl.metadata (2.2 kB)
  U

In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from ibm_watsonx_ai import Credentials
from langchain_ibm import WatsonxLLM, WatsonxEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from huggingface_hub import HfFolder
import gradio as gr
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [19]:
## llm

def get_llm():
    model_id = 'mistralai/mistral-small-3-1-24b-instruct-2503'
    parameters = {
        "temperature":  0.5,
        "max_new_tokens": 256,
    }
    project_id = "skills-network"
    watsonx_llm = WatsonxLLM(
        model_id=model_id,
        url= "https://us-south.ml.cloud.ibm.com",
        project_id=project_id,
        params=parameters,
    )
    return watsonx_llm

In [20]:
## Document loader

def document_loader(file):
    loader = PyPDFLoader(file.name)
    loaded_document=loader.load()
    return loaded_document

In [21]:
# text_spliiter

def text_splitter(data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len
    )
    chunks = text_splitter.split_documents(data)
    return chunks

In [22]:
# Embedding model

def watsonx_embedding():
    embed_params = {
        "max_new_tokens":5
    },

    watsonx_embedding = WatsonxEmbeddings(
        model_id = "ibm/slate-125-english-embeddings",
        url="https://us-south.ml.cloud.ibm.com",
        project_id="skills-network",
        params="embed_params"
    )
    return watsonx_embedding

In [23]:
# vector db
def vector_database(chunks):
    embedding_model = watsonx_embedding()
    vectordb = Chroma.from_documents(
        chunks,
        embedding_model
    )
    return vectordb

In [24]:
# retriever
def retriever(file):
    splits=document_loader(file)
    chunks=text_splitter(splits)
    vectordb=vector_database(chunks)
    retriever=vectordb.as_retriever()
    return retriever

In [25]:
# QA Chain
def retriever_qa(file, query):
    llm = get_llm()
    retriever_obj = retriever(file)
    qa = RetrievalQA.from_chain_type(llm=llm,
                         chain_type="stuff",
                         retriever=retriever_obj,
                                     return_source_documents=True)

    response=qa.invoke(query=query)
    return response['result']

In [26]:
# Create Gradio interface
rag_application = gr.Interface(
    fn=retriever_qa,
    allow_flagging="never",
    inputs=[
        gr.File(label="Upload PDF File", file_count="single", file_types=['.pdf'], type="filepath"),  # Drag and drop file upload
        gr.Textbox(label="Input Query", lines=2, placeholder="Type your question here...")
    ],
    outputs=gr.Textbox(label="Output Query"),
    title="Watsonx PDF QA Bot",
    description="Upload a PDF document and ask any question. The chatbot will try to answer using the provided document."
)



In [27]:
# launch the app
rag_application.launch(server_name="0.0.0.0", server_port=7876)

Running on local URL:  http://0.0.0.0:7876

To create a public link, set `share=True` in `launch()`.
